# Install the requirements

In [1]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 366.3 kB/s eta 0:00:001m374.2 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77717 sha256=94d4e52e6cb37dcee8d68064824d6ad47c12f504d0cc75f17d380d3634bb1808
  Stored in directory: /home/elcan/.cache/pip/wheels/9c/45/15/6d6d116cd2539fb8f450d64b0aee4a480e5366bb11b42ac763
Successfully built kaggle


In [2]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [1]:
from os import environ 
from dotenv import load_dotenv
load_dotenv()

True

# Download the dataset via Kaggle API

In [2]:
environ.get("KAGGLE_CONFIG_DIR")

'/home/elcan/Downloads/'

In [7]:
!kaggle datasets download -d peopledatalabssf/free-7-million-company-dataset

100%|█████████████████████████████████████████| 402k/402k [00:00<00:00, 565kB/s]
100%|█████████████████████████████████████████| 402k/402k [00:00<00:00, 564kB/s]


In [10]:
# move the downloaded .zip file to the data/ folder 
!mv free-7-million-company-dataset.zip ../data/

In [11]:
# unzip the filet
import zipfile
with zipfile.ZipFile('../data/free-7-million-company-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('../data/')

# Import the dataset

In [1]:
import pandas as pd

In [2]:
companies = pd.read_csv("../data/companies_sorted.csv")

In [3]:
companies.shape

(7173426, 11)

In [4]:
companies.head(3)

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768


# Data Cleaning

In [5]:
# lower and replace the whitespace with underscore
companies.columns = companies.columns.str.lower()
companies = companies.rename(columns=lambda x: x.replace(' ', '_'))

In [6]:
companies.columns

Index(['unnamed:_0', 'name', 'domain', 'year_founded', 'industry',
       'size_range', 'locality', 'country', 'linkedin_url',
       'current_employee_estimate', 'total_employee_estimate'],
      dtype='object')

In [7]:
# dropping redundant columns
companies.drop(columns=[
  "unnamed:_0", "domain", "year_founded", "industry", "size_range", "locality", 
  "linkedin_url", "total_employee_estimate"], axis=1, inplace=True
              )

In [8]:
companies.name.isnull().sum()

3

In [9]:
# drop null values from `name` column
companies.dropna(subset=["name"], inplace=True)

In [10]:
# remove duplicates
companies.name = companies.name.apply(lambda x: x.lower())
companies.drop_duplicates(subset=["name"], keep="first", inplace=True)

In [11]:
companies.shape

(7004634, 3)

In [79]:
companies_copy[companies_copy.name == "spacex"]

,name,country,current_employee_estimate
2511,spacex,united states,3583


In [13]:
# for the sake of simplicity only filter those with single word only
companies = companies[companies.name.apply(lambda x: len(x.split()) == 1)]

In [14]:
companies.shape

(914701, 3)

In [15]:
# reducing the shape further setting country to US only for running experiments faster
  # may include those filtered names in the future
companies = companies[companies.country == "united states"]

In [16]:
companies[(companies.country == "united states") 
         & (companies.name.str.contains("com"))]

,name,country,current_employee_estimate
173,aecom,united states,26008
184,comcast,united states,24515
255,qualcomm,united states,18614
994,broadcom,united states,7281
1416,viacom,united states,5535
...,...,...,...
7171644,wisgo.com,united states,0
7171822,activmedias.com,united states,0
7171883,letslunch.com,united states,0
7171998,employeebenchmark.com,united states,0


In [17]:
companies_copy = companies.copy()

In [18]:
# remove ".com" from the names
companies.name = companies.name.apply(lambda x: x.replace('.com', ''))

In [19]:
companies[(companies.country == "united states") 
         & (companies.name.str.contains(".", regex=False))]

,name,country,current_employee_estimate
2025,j.crew,united states,4245
5745,l.l.bean,united states,1840
6338,[24]7.ai,united states,1686
11524,j.hilburn,united states,978
17259,odyssey.,united states,672
...,...,...,...
7166645,inkfx.tattoo,united states,0
7169694,urgift.in,united states,0
7170926,helpdesk.net,united states,0
7171447,mustbeonline.net,united states,0


In [20]:
# remove names that contains character other than english alphabet
companies = companies[~companies.name.str.contains(r'[^a-zA-Z\s]')]
companies.shape

(126401, 3)

In [40]:
min(len(c) for c in companies.name.to_list()) # minimum string

2

In [58]:
max(len(c) for c in companies.name.to_list()) # maximum string

24

In [66]:
companies = companies[companies.name.str.len() > 1] # delete names with length less than 1
companies = companies[companies.name.str.len() < 21] # and more than 20

In [80]:
companies

,name,country,current_employee_estimate
0,ibm,united states,274047
7,walmart,united states,120753
8,microsoft,united states,116196
11,pwc,united states,111372
14,deloitte,united states,104112
...,...,...,...
7172838,hykoo,united states,0
7173079,paulfirth,united states,0
7173103,beamaco,united states,0
7173131,madminds,united states,0


In [83]:
companies.reset_index(inplace=True, drop=True)
companies.head(2)

,name,country,current_employee_estimate
0,ibm,united states,274047
1,walmart,united states,120753


In [87]:
# ended up with the 126 021 unique company names
  # it is likely to remove some filters to have more names in the later stages of the research

In [86]:
# export the cleaned dataset
companies.to_csv("../data/companies_cleaned.csv", index=False)